In [1]:
import json

In [4]:
import pprint

In [7]:
with open('data') as json_data:
    d = json.load(json_data)
    pprint.pprint(d)

{'8XBfdAwp1Y_IrhFdSZaNzw': [['The customer service here overall is better than '
                             'any other store over here. Especially the '
                             'Verizon store which has horrible customer '
                             'service. \xa0Crystal, the manager there, is '
                             "awesome. She's very good at what she does and "
                             'understands true customer service. But the wait '
                             'time here is very very long. Be prepared to wait '
                             'at least 30-40 minutes for help!',
                             3.0],
                            ['One point for swiftly getting the paperwork '
                             "done.That's it.Unfortunately, I trusted Miss "
                             'Lulu K. that there were no additional fees.I '
                             'explicitly asked: "Ok. I\'ll pay the activation '
                             'fee of $43.20, b

                             'in my car if I was parked close enough. Good job '
                             'guys. Thank you.',
                             5.0],
                            ['This location has fantastic customer service. '
                             'The team members are always friendly & very '
                             'efficient. I have been coming here for 3 years '
                             'and its always a pleasure!',
                             5.0],
                            ['Abel has the worst customer service skills EVER! '
                             'My husband and I were pulling teeth to even get '
                             'one-worded answers out of him. We will not be '
                             'back to this store.',
                             1.0],
                            ['This has to be my favorite T Mobile location. '
                             'The customer service is beyond awesome. They '
                           